# Christoffel Symbols
The Christoffel symbols of the first kind are defined as
$$
\Gamma_{ij|k} = \frac{1}{2} \left( \frac{\partial g_{kj}}{\partial x^i} + \frac{\partial g_{ik}}{\partial x^j} - \frac{\partial g_{ij}}{\partial x^k} \right)
$$
Vertical bar is to divide the sym part from the other part.


In [ ]:
from ngsolve import *
from zenith import * 
from ngsolve import *
from netgen.csg import *
import scipy.sparse as sp
import matplotlib.pylab as plt



# import netgen.webgui  
# from ngsolve.webgui import Draw
import netgen.gui
%gui tk

In [ ]:
# create an index function that maps a symmetric matrix 3 x 3 to a vector of length 6
# the indices are mapped as follows: i<=j
# 0 -> 0,0
# 1 -> 0,1
# 2 -> 0,2
# 3 -> 1,1
# 4 -> 1,2
# 5 -> 2,2


def Idx(i, j,k =0):
    if i>j: i,j = j,i
    return i*3+j-(i*(i+1))//2 + k*6


# print the indices as 3 matrices 
for i in range(3):
    for j in range(3):
        print(Idx(i,j), end=" ")
    print() 

# print the indices as 3 matrices
for i in range(3):
    for j in range(3):
        print(Idx(i,j,1), end=" ")
    print()

# print the indices as 3 matrices
for i in range(3):
    for j in range(3):
        print(Idx(i,j,2), end=" ")
    print()

In [ ]:
# standard mesh
with TaskManager():
    # create a mesh
    h = 0.2
    r = 0.5
    H = 0.4
    R = 1
    order = 3
    kwargs = {"bonus_intorder": 10, "inverse": "pardiso", "order": order, "mesh_order": 1}

    # BH1
    pos1 =  (0,0,0)
    lin1 = (0,0,0)
    ang1 = (0,0,0)
    mass1 =1
    BH1 = BlackHole(mass1, pos1, lin1, ang1 )

    ## BH2
    #pos2 =  (-1,0,0)
    #lin2 = (0,-1,0)
    #ang2 = (0,0,1)
    #mass2 =1
    #BH2 = BlackHole(mass2, pos2, lin2, ang2 )

    BHs = [BH1]##, BH2]

    mesh = MeshBlackHoles(BHs, h=h, R=R, H = H, r= r,  curve_order = 3)

Draw(mesh, clipping= {"z":-1})

In [ ]:
scl_h1 = H1(mesh, order=order)#, dirichlet="outer")
mat_h1 = VectorValued(scl_h1, 6)
tns_h1 = VectorValued(scl_h1, 18) # it is 6 x 3 

scl, dscl = scl_h1.TnT()
m, dm = mat_h1.TnT()
t, dt = tns_h1.TnT()

# define the evolution variables
gf_g  = GridFunction(mat_h1)
gf_G  = GridFunction(tns_h1)



# set a positive inital condition all the variables
def Peak(pnt = (0,0,0) ,**kwargs) : return exp(-kwargs.get("sigma", 1)* ( (x-pnt[0])**2 + (y-pnt[1])**2 + (z-pnt[2])**2 ) )
peak = Peak(sigma = 10)

gf_g.Set( (1+ peak,0,0,1+ peak, 0 , 1+ peak)  , bonus_intorder = 10)
Draw(gf_g , mesh, "g", clipping= {"z":-1})



fes = gf_g.space * gf_G.space 

#gf_out = GridFunction(fes)
gf_in = GridFunction(fes)

# work only with components 0, 5
gf_in.components[0].vec.data = gf_g.vec

In [ ]:
# use embedding to create inverse of whole matrix
inverse = "sparsecholesky"

with TaskManager():
    ## g
    mass_g = BilinearForm(mat_h1, symmetric=True)
    mass_g += InnerProduct(m, dm)*dx
    mass_g.Assemble()
    mass_g_inv = mass_g.mat.Inverse(inverse=inverse)
    res_g = fes.restrictions[0]
    inv_g = res_g.T@mass_g_inv@res_g

    ## G
    mass_G = BilinearForm(tns_h1, symmetric=True)
    mass_G += InnerProduct(t, dt)*dx
    mass_G.Assemble()
    mass_G_inv = mass_G.mat.Inverse(inverse=inverse)
    res_G = fes.restrictions[1]
    inv_G = res_G.T@mass_G_inv@res_G


In [ ]:

# def CS1(g):
#     gradg = grad(g)
#     cs = 0.5*( fem.Einsum("ijk->ikj", gradg) + fem.Einsum("ikj->jki", gradg) - fem.Einsum("jik->ijk", gradg) )
#     return cs


In [ ]:

# trial and test functions
g,  G = fes.TrialFunction()
dg, dG = fes.TestFunction()

chris1 = CF( (  dG[0+6*0] , dG[1+6*0], dG[2+6*0], 
                dG[1+6*0] , dG[3+6*0], dG[4+6*0], 
                dG[2+6*0], dG[4+6*0], dG[5+6*0] ,
                dG[0+6*1] , dG[1+6*1], dG[2+6*1],
                dG[1+6*1] , dG[3+6*1], dG[4+6*1],
                dG[2+6*1], dG[4+6*1], dG[5+6*1] ,
                dG[0+6*2] , dG[1+6*2], dG[2+6*2],
                dG[1+6*2] , dG[3+6*2], dG[4+6*2],
                dG[2+6*2], dG[4+6*2], dG[5+6*2]
              ), dims=(3,3,3) )
              
blf = BilinearForm(fes)
gradg = grad(g)
for i in [0,1,2]:
    for j in [0,1,2]:
        for k in [0,1,2]:
            blf += 0.5*(InnerProduct(gradg[Idx(k,j),i],chris1[i,j,k]) + InnerProduct(gradg[Idx(k,i),j],chris1[i,j,k]) - InnerProduct(gradg[Idx(i,j),k],chris1[i,j,k]))*dx
blf.Assemble()



In [ ]:
A = sp.csr_matrix(blf.mat.CSR())
plt.rcParams['figure.figsize'] = (10,10)
plt.spy(A)
plt.show()

In [ ]:
inv_mass = inv_g +  inv_G 


In [ ]:
gf_out = GridFunction(fes)

with TaskManager():
        
    gf_out.vec.data = inv_mass @ blf.mat *gf_in.vec



    


In [ ]:
Draw(gf_out.components[1], mesh, "Gamma_out",  clipping= {"z":-1})



In [ ]:
True_g = CF((1+ peak,0,0,0,1+ peak, 0,0,0 , 1+ peak), dims=(3,3) )
True_grad_g = CF((True_g.Diff(x), True_g.Diff(y), True_g.Diff(z)), dims=(3,3,3) )
True_Gamma = 0.5*(fem.Einsum("ijk->ikj", True_grad_g) + fem.Einsum("ijk->jki", True_grad_g) - fem.Einsum("ijk->ijk", True_grad_g))

In [ ]:
for i in [0,1,2]:
    for j in [0,1,2]:
        for k in [0,1,2]:
           Draw(True_Gamma[i,j,k], mesh, "Gamma_"+str(i)+str(j)+str(k) ,  clipping= {"z":-1})
           input(str(i)+str(j)+str(k) )

In [ ]:

#Draw(gf_out.components[1][] , mesh, "Gamma_comp",  clipping= {"z":-1})

In [ ]:
i = 0
j = 0
k = 0

Draw(gf_out.components[1][Idx(i,j,k)], mesh, "Gamma" ,  clipping= {"z":-1})


In [ ]:
Draw(True_Gamma[i,j,k], mesh, "Gamma" ,  clipping= {"z":-1})


In [ ]:
Draw(True_Gamma[i,j,k]-gf_out.components[1][Idx(i,j,k)], mesh, "Gamma" ,  clipping= {"z":-1})


In [ ]:
tot =0.0
for i in [0,1,2]:
    for j in [0,1,2]:
        for k in [0,1,2]:
            tot += Integrate( Norm(True_Gamma[i,j,k]-gf_out.components[1][Idx(i,j,k)]) , mesh)



In [ ]:
print(tot)

In [ ]:
E000 = True_Gamma[0,0,0]-gf_out.components[1][Idx(0,0,0)]
E001 = True_Gamma[0,0,1]-gf_out.components[1][Idx(0,0,1)]
E002 = True_Gamma[0,0,2]-gf_out.components[1][Idx(0,0,2)]
E010 = True_Gamma[0,1,0]-gf_out.components[1][Idx(0,1,0)]
E011 = True_Gamma[0,1,1]-gf_out.components[1][Idx(0,1,1)]
E012 = True_Gamma[0,1,2]-gf_out.components[1][Idx(0,1,2)]
E020 = True_Gamma[0,2,0]-gf_out.components[1][Idx(0,2,0)]
E021 = True_Gamma[0,2,1]-gf_out.components[1][Idx(0,2,1)]
E022 = True_Gamma[0,2,2]-gf_out.components[1][Idx(0,2,2)]
E100 = True_Gamma[1,0,0]-gf_out.components[1][Idx(1,0,0)]
E101 = True_Gamma[1,0,1]-gf_out.components[1][Idx(1,0,1)]
E102 = True_Gamma[1,0,2]-gf_out.components[1][Idx(1,0,2)]
E110 = True_Gamma[1,1,0]-gf_out.components[1][Idx(1,1,0)]
E111 = True_Gamma[1,1,1]-gf_out.components[1][Idx(1,1,1)]
E112 = True_Gamma[1,1,2]-gf_out.components[1][Idx(1,1,2)]
E120 = True_Gamma[1,2,0]-gf_out.components[1][Idx(1,2,0)]
E121 = True_Gamma[1,2,1]-gf_out.components[1][Idx(1,2,1)]
E122 = True_Gamma[1,2,2]-gf_out.components[1][Idx(1,2,2)]
E200 = True_Gamma[2,0,0]-gf_out.components[1][Idx(2,0,0)]
E201 = True_Gamma[2,0,1]-gf_out.components[1][Idx(2,0,1)]
E202 = True_Gamma[2,0,2]-gf_out.components[1][Idx(2,0,2)]
E210 = True_Gamma[2,1,0]-gf_out.components[1][Idx(2,1,0)]
E211 = True_Gamma[2,1,1]-gf_out.components[1][Idx(2,1,1)]
E212 = True_Gamma[2,1,2]-gf_out.components[1][Idx(2,1,2)]
E220 = True_Gamma[2,2,0]-gf_out.components[1][Idx(2,2,0)]
E221 = True_Gamma[2,2,1]-gf_out.components[1][Idx(2,2,1)]
E222 = True_Gamma[2,2,2]-gf_out.components[1][Idx(2,2,2)]

Error = CF((E000,E001,E002,E010,E011,E012,E020,E021,E022,E100,E101,E102,E110,E111,E112,E120,E121,E122,E200,E201,E202,E210,E211,E212,E220,E221,E222), dims=(3,3,3) )

In [ ]:
Draw(Error, mesh, "Error" ,  clipping= {"z":-1})